ESE P6: An Empirical Study of GitHub code review tool 
by Eleonora Pura  and Melih Catal


## Data Mining
This step is skipped because the data is already ready and given by the tutor.

## Methods Extraction and Linking Reviewer Comment
Parsing Java files using the Lizard Python library to extract the methods. We are only interested in the java files.
Link each comment to the specific method (if any) it refers to. If a comment cannot be linked to any method, it is discarded. After having linked comments to methods for each review round, we are in the situation in which we have, for each review round, a set of triplets <ms,mr and {Rnl}> where ms and mr represent the same method before and after the review round, and Rnl is a set of comments ms received in this round.

In [1]:
import lizard
import pandas as pd
import numpy as np
import subprocess
import re
import nltk
nltk.download('stopwords')  # download the stop words corpus
from nltk.corpus import stopwords



[nltk_data] Downloading package stopwords to /Users/melih/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def find_function_body(source_code, start_line, end_line):
    lines = source_code.split('\n')[start_line-1: end_line+1]
    return "\n".join(lines)


# TODO: check if comment_start_line and comment_end line are the correct ones
def comment_in_method_body(comment_start_line, comment_end_line, function_start_line, function_end_line):
    comment_range = set(range(comment_start_line, comment_end_line+1))
    function_range = set(range(function_start_line, function_end_line+1))
    
    if comment_range.issubset(function_range):
        return True
    else:
        return False
    
# TODO: fix function, not working yet
def remove_unchanged_functions(functions_while, functions_after):
    if functions_while == [] or functions_after == []:
        return functions_while, functions_after
    
    for index, (function_while, function_after) in enumerate(zip(functions_while, functions_after)):
        if function_while == function_after:
            del functions_while[index]
            del functions_after[index]
            
    return functions_while, functions_after

def is_reviewer_commenting(owner_id, user_id):
    if owner_id != user_id:
        return True
    
    else:
        return False
    
    

In [3]:
df = pd.read_csv('./data/gh_apache_!_accumulo.csv')
df.head()


,project,url,discussion,pull_number,pull_id,filename,start_line,original_start_line,start_side,side,...,updated_at,commit_before,commit_while,commit_after,url_source_before,url_source_while,url_source_after,file_content_before,file_content_while,file_content_after
0,apache/accumulo,https://github.com/apache/accumulo/pull/1622,https://github.com/apache/accumulo/pull/1622#d...,1622,428010670,core/src/main/java/org/apache/accumulo/core/co...,NaN,NaN,NaN,RIGHT,...,2020-06-05T12:44:03Z,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,4d6991afc9c9589b829c2b9677a47e85586cc5aa,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
1,apache/accumulo,https://github.com/apache/accumulo/pull/1622,https://github.com/apache/accumulo/pull/1622#d...,1622,428010670,core/src/main/java/org/apache/accumulo/core/co...,NaN,NaN,NaN,RIGHT,...,2020-06-05T12:44:03Z,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,4d6991afc9c9589b829c2b9677a47e85586cc5aa,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
2,apache/accumulo,https://github.com/apache/accumulo/pull/1622,https://github.com/apache/accumulo/pull/1622#d...,1622,428010670,core/src/main/java/org/apache/accumulo/core/co...,NaN,NaN,NaN,RIGHT,...,2020-06-05T12:44:03Z,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,4d6991afc9c9589b829c2b9677a47e85586cc5aa,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
3,apache/accumulo,https://github.com/apache/accumulo/pull/1622,https://github.com/apache/accumulo/pull/1622#d...,1622,428010670,core/src/main/java/org/apache/accumulo/core/co...,NaN,NaN,NaN,RIGHT,...,2020-06-05T12:44:03Z,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,4d6991afc9c9589b829c2b9677a47e85586cc5aa,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
4,apache/accumulo,https://github.com/apache/accumulo/pull/1614,https://github.com/apache/accumulo/pull/1614#d...,1614,420303336,test/src/main/java/org/apache/accumulo/test/fu...,NaN,NaN,NaN,RIGHT,...,2020-06-04T19:21:57Z,13ad1b33bc3e92669db8a523e1e3a87025326d7a,ab92f60d0687aa9e3be8b0637240c088739b71fa,6319d9e408350cd15570eb9a93de613e5d4e730e,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...


In [4]:
# Extract all the function from a row of the df
def extract_row_functions(row, file_content):
    functions = []
    # Taken from Melih's code:
    # if the comment start line is na that means the comment is a single line comment so we use the end line number as the start line number
    comment_start_line = row['original_start_line'] if not pd.isna(row['original_start_line']) else row['original_line']
    comment_end_line = row['original_line']
    # Check that the comment is given by a reviewer
    if is_reviewer_commenting(row['owner_id'], row['user_id']):
        # Check that both start and end line are not NaN floats
        if (not pd.isna(comment_start_line)) and (not pd.isna(comment_end_line)):
            # Convert all start and end line to int 
            comment_start_line = int(comment_start_line)
            comment_end_line = int(comment_end_line)

            
            l = lizard.analyze_file.analyze_source_code(row['filename'], row[file_content])
            l_functions = l.function_list

            for function in l_functions:
                function_start_line = int(function.start_line)
                function_end_line = int(function.end_line)
                
                # Check that the comment is part of the body of the function
                if comment_in_method_body(comment_start_line, comment_end_line, function_start_line, function_end_line):
                    body = find_function_body(row[file_content], function_start_line, function_end_line)
                    
                    functions.append(
                        {"name": function.name,
                         "long_name": function.long_name,
                         "start_line": int(function.start_line),
                         "end_line": int(function.end_line),
                         "body": body})
    return functions
   

In [5]:
# Make a copy of the original df
df_copy = df.copy()

In [6]:
functions_while = []
functions_after = []
for idx, row in df_copy.iterrows():
    row_functions_while = extract_row_functions(row, 'file_content_while')
    row_functions_after = extract_row_functions(row, 'file_content_after')
    
    
    """
    #Remove the functions that remained unchanged
    row_functions_while, row_functions_after = remove_unchanged_functions(functions_while, functions_after)
    """
    
    functions_while.append(row_functions_while)
    functions_after.append(row_functions_after)



In [7]:
# Add 2 new columns to df: functions_while and functions_after
df_copy['functions_while'] = functions_while
df_copy['functions_after'] = functions_after

# Set all empty list values as np.nan in order to use pandas dropna function
df_copy.functions_while = df_copy.functions_while.apply(lambda y: np.nan if len(y)==0 else y)
df_copy.functions_after = df_copy.functions_after.apply(lambda y: np.nan if len(y)==0 else y)

# Delete all entries in which there is no functions while or after
df_copy = df_copy.dropna(subset=['functions_while', 'functions_after']).reset_index(drop=True)


# If the functions while and after remains the same that means that no error was fixed: remove from df
df_copy = df_copy[df_copy['functions_while'] != df_copy['functions_after']].reset_index(drop=True)


In [8]:
df_copy.head()

,project,url,discussion,pull_number,pull_id,filename,start_line,original_start_line,start_side,side,...,commit_while,commit_after,url_source_before,url_source_while,url_source_after,file_content_before,file_content_while,file_content_after,functions_while,functions_after
0,apache/accumulo,https://github.com/apache/accumulo/pull/1614,https://github.com/apache/accumulo/pull/1614#d...,1614,420303336,test/src/main/java/org/apache/accumulo/test/fu...,NaN,NaN,NaN,RIGHT,...,ab92f60d0687aa9e3be8b0637240c088739b71fa,6319d9e408350cd15570eb9a93de613e5d4e730e,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,"[{'name': 'BulkNewIT::testMaxTablets', 'long_n...","[{'name': 'BulkNewIT::testMaxTablets', 'long_n..."
1,apache/accumulo,https://github.com/apache/accumulo/pull/1614,https://github.com/apache/accumulo/pull/1614#d...,1614,420303336,core/src/main/java/org/apache/accumulo/core/cl...,NaN,NaN,NaN,RIGHT,...,ab92f60d0687aa9e3be8b0637240c088739b71fa,6319d9e408350cd15570eb9a93de613e5d4e730e,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,"[{'name': 'BulkImport::load', 'long_name': 'Bu...","[{'name': 'BulkImport::load', 'long_name': 'Bu..."
2,apache/accumulo,https://github.com/apache/accumulo/pull/1614,https://github.com/apache/accumulo/pull/1614#d...,1614,420303336,test/src/main/java/org/apache/accumulo/test/fu...,NaN,NaN,NaN,RIGHT,...,ab92f60d0687aa9e3be8b0637240c088739b71fa,6319d9e408350cd15570eb9a93de613e5d4e730e,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,"[{'name': 'BulkNewIT::testMaxTablets', 'long_n...","[{'name': 'BulkNewIT::testMaxTablets', 'long_n..."
3,apache/accumulo,https://github.com/apache/accumulo/pull/1614,https://github.com/apache/accumulo/pull/1614#d...,1614,420303336,test/src/main/java/org/apache/accumulo/test/fu...,NaN,NaN,NaN,RIGHT,...,ab92f60d0687aa9e3be8b0637240c088739b71fa,6319d9e408350cd15570eb9a93de613e5d4e730e,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,"[{'name': 'BulkNewIT::testMaxTablets', 'long_n...","[{'name': 'BulkNewIT::testMaxTablets', 'long_n..."
4,apache/accumulo,https://github.com/apache/accumulo/pull/1614,https://github.com/apache/accumulo/pull/1614#d...,1614,420303336,core/src/main/java/org/apache/accumulo/core/cl...,NaN,NaN,NaN,RIGHT,...,ab92f60d0687aa9e3be8b0637240c088739b71fa,6319d9e408350cd15570eb9a93de613e5d4e730e,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,"[{'name': 'BulkImport::load', 'long_name': 'Bu...","[{'name': 'BulkImport::load', 'long_name': 'Bu..."


In [9]:
# Turn the dataframe in a list of triplets: (function_while, comment, function_after)
triplets = []

for idx, row in df_copy.iterrows():
    if len(row['functions_while']) == len(row['functions_after']):
        for function_while, function_after in zip(row['functions_while'], row['functions_after']):
            triplets.append((function_while['body'], row['message'], function_after['body']))

In [10]:
print(f"Function while:\n{triplets[0][0]}\n")
print(f"Review comment:\n{triplets[0][1]}\n")
print(f"Function after:\n{triplets[0][2]}\n")

Function while:
  public void testMaxTablets() throws Exception {
    String maxTablets = "0";
    try (AccumuloClient client = Accumulo.newClient().from(getClientProps()).build()) {
      maxTablets = client.instanceOperations().getSystemConfiguration()
          .get(Property.MASTER_BULK_MAX_TABLETS.getKey());
      client.instanceOperations().setProperty(Property.MASTER_BULK_MAX_TABLETS.getKey(), "1");
      testBulkFile(false, false);
      fail("Expected IllegalArgumentException for " + Property.MASTER_BULK_MAX_TABLETS);
    } catch (IllegalArgumentException e) {} finally {
      try (AccumuloClient client = Accumulo.newClient().from(getClientProps()).build()) {
        client.instanceOperations().setProperty(Property.MASTER_BULK_MAX_TABLETS.getKey(),
            maxTablets);
      }
    }
  }


Review comment:
Maybe just a style comment - but I don't think this needs to be in a finally block.  If exception is not thrown, it will fail - otherwise catch (and ignore the exception) a

In [11]:
print(len(triplets))

1666


## Abstraction
Use `src2abs` tool to abstract the methods. Triplets for which a parsing error occur during the abstraction process on the ms or on the mr methods are removed from the dataset. ms and mr, after the abstraction must be different. Idioms are not abstracted. During the abstraction code comments are removed. Using the pair abstraction mode, the same literals/identifiers in the two methods weill be abstracted using the same IDs. As output of the abstract process, the tool provides an abstraction map M linking the abstracted token to the raw token.

In [12]:
def create_java_file(content, filename):
    path = f"./abstraction/java_files/{filename}"
    with open(path, 'w') as f:
        f.write(content)

In [13]:
def create_abstraction_file():
    code_granularity = "method"
    mode = "pair"
    idioms_path = "./src2abs/idioms.csv"
    src2abs_path = "./src2abs/src2abs-0.1-jar-with-dependencies.jar"
    input_code_A_path = "./abstraction/java_files/ms.java"
    input_code_B_path = "./abstraction/java_files/mr.java"
    output_abstract_A_path = "./abstraction/abstraction_files/ms_abs.java"
    output_abstract_B_path = "./abstraction/abstraction_files/mr_abs.java"
    try :
        # java -jar src2abs-0.1-jar-with-dependencies.jar pair <code_granularity> <input_code_A_path> <input_code_B_path> <output_abstract_A_path> <output_abstract_B_path> <idioms_path>
        command = ['java', '-jar', src2abs_path, mode, code_granularity, input_code_A_path, input_code_B_path, output_abstract_A_path, output_abstract_B_path, idioms_path]
        # execute the command and capture the output
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        if stderr:
            print("Error creating abstraction data")
            print(stderr)
        #else :
            #print("Abstraction data created")
    except Exception as e:
        print("Error creating abstraction data")
        print(e)

In [14]:
def read_abstraction_file(filename):
    path = f"./abstraction/abstraction_files/{filename}"
    with open(path, 'r') as f:
        return f.read()

In [15]:
# read mapping file and create a dictionary with the mapping. key: concrete, value: abstract
def read_abstraction_mapping_file(filename):
    path = f"./abstraction/abstraction_files/{filename}"
    with open(path, 'r') as f:
        lines = f.readlines()
    is_abstract = False
    keys = []
    values = []
    for line in lines:
        words = line.strip().split(',')
        if len(words) > 1 :
            for word in words:
                if word == '':
                    continue
                if is_abstract:
                    values.append(word)
                else:
                    keys.append(word)
            is_abstract = not is_abstract
     # convert the lists to a dictionary
    maps = dict(zip(keys, values))


    return maps

In [16]:
'''
template_triplets_data = {
    "ms": "",
    "comment": "",
    "mr": ""
    "ms_abs": "",
    "mr_abs": "",
    "map" : dict()
}
ms = methods while
mr = methods after change
rnl = review comment
'''
triplets_with_abstraction = []
def create_abstraction_data(ms,mr,rnl):
    template_triplets_data = {}
    create_java_file(ms, "ms.java")
    create_java_file(mr, "mr.java")
    create_abstraction_file()
    ms_abs = read_abstraction_file("ms_abs.java")
    mr_abs = read_abstraction_file("mr_abs.java")
    mapping = read_abstraction_mapping_file("ms_abs.java.map")
    
    template_triplets_data["ms"] = ms
    template_triplets_data["comment"] = rnl
    template_triplets_data["mr"] = mr
    template_triplets_data["ms_abs"] = ms_abs
    template_triplets_data["mr_abs"] = mr_abs
    template_triplets_data["map"] = mapping
    triplets_with_abstraction.append(template_triplets_data)



    

In [17]:
for triplet in triplets:
    create_abstraction_data(triplet[0], triplet[2], triplet[1])

print(len(triplets_with_abstraction))

Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creating abstraction data
b'Parsing ERROR!\n'
Error creati

## Abstracting Reviewer Comments 
Abstract all code components mentioned in any comment using the abstraction map. Any camel casse identifier that is not matched in the abstraction map but that it is present in the comment, is replaced by the special token _CODE_ 

In [18]:
def is_camel_case(s):
    # example: "camelCase", "camelcase", "camelCaseCase", "camelCaseC"
    # not example: "camel", "camelCase_"
    pattern = r'^(?:[A-Z][a-z]*|[a-z]+)((?:[A-Z][a-z]*))+$'
    return bool(re.match(pattern, s))

def remove_stop_words(comment):
    stop_words = set(stopwords.words('english'))
    words = comment.split()
    filtered_comment = [w for w in words if not w in stop_words]
    return " ".join(filtered_comment)
    
def abstract_reviewer_comments(triplets_with_abstraction):
    camel_case_abs = "_CODE_"
    for triplet in triplets_with_abstraction:
        comment = triplet["comment"]
        mapping = triplet["map"]
        comment_words = comment.split()
        for word in comment_words:
            # remove any special characters from the word
            word = re.sub(r'[^\w\s]', '', word)
            if word in mapping:               
                comment = comment.replace(word, mapping[word])
            elif is_camel_case(word):
                comment = comment.replace(word, camel_case_abs)
        comment = remove_stop_words(comment)
        triplet["comment_abs"] = comment

    return triplets_with_abstraction
    

In [19]:
triplets_with_abstraction = abstract_reviewer_comments(triplets_with_abstraction)



In [20]:
print(triplets_with_abstraction[0]['comment'])
print(triplets_with_abstraction[0]['comment_abs'])
print(triplets_with_abstraction[0]['map'])


Maybe just a style comment - but I don't think this needs to be in a finally block.  If exception is not thrown, it will fail - otherwise catch (and ignore the exception) and then just continue.
Maybe style comment - I think needs finally block. If exception VAR_7, fail - otherwise catch (and ignore exception) continue.
{'AccumuloClient': 'TYPE_2', 'Accumulo': 'TYPE_3', 'NewTableConfiguration': 'TYPE_6', 'Property.TABLE_BULK_MAX_TABLETS': 'TYPE_8', 'ExecutionException': 'TYPE_9', 'Map': 'TYPE_7', 'Exception': 'TYPE_1', 'IllegalArgumentException': 'TYPE_5', 'Property.MASTER_BULK_MAX_TABLETS': 'TYPE_4', 'testMaxTablets': 'METHOD_1', 'getKey': 'METHOD_7', 'tableOperations': 'METHOD_12', 'setProperties': 'METHOD_11', 'getClientProps': 'METHOD_4', 'testBulkFileMax': 'METHOD_15', 'getMessage': 'METHOD_17', 'getCause': 'METHOD_16', 'newClient': 'METHOD_2', 'testBulkFile': 'METHOD_9', 'getSystemConfiguration': 'METHOD_6', 'setProperty': 'METHOD_8', 'of': 'METHOD_10', 'create': 'METHOD_13', 'fr

## Filtering out Noisy Comments
Questions: 
1. Should we create a new dataset with the manually labeled comments as relevant or not?
2. Should we use the heuristic approach provided in the paper?


In [21]:
print(len(triplets_with_abstraction))

1666


In [22]:
# shuffle the data
import random
random.shuffle(triplets_with_abstraction)
# select 80% of the data for training
train_size = int(len(triplets_with_abstraction) * 0.7)
train_data = triplets_with_abstraction[:train_size]
test_data = triplets_with_abstraction[train_size:]

print(len(train_data))


1166


In [24]:
# for each training data print the comment and the ms 
for data in train_data:
    print(data['comment'])
    print(data['ms'])
    print("")
    break

The comment does not accurately describe the behavior of this API call. This doesn't remove any watches. By passing "false" here, it simply doesn't add any new ones.
  public void removeTable(TableId tableId) throws KeeperException, InterruptedException {
    synchronized (tableStateCache) {
      tableStateCache.remove(tableId);
      zoo.recursiveDelete(zkRoot + Constants.ZTABLES + "/" + tableId + Constants.ZTABLE_STATE,
          NodeMissingPolicy.SKIP);
      String tableConfigPath = zkRoot + Constants.ZTABLES + "/" + tableId + Constants.ZTABLE_CONF;
      // The line below removes all the watches on the table configuration items from the Zookeeper
      // server
      zoo.getZooKeeper().getChildren(tableConfigPath, false);
      zoo.recursiveDelete(zkRoot + Constants.ZTABLES + "/" + tableId, NodeMissingPolicy.SKIP);
    }
  }




In [25]:
# for each training data print the comment and the ms 
for data in train_data:
    print(data['comment'])
    print(data['comment_abs'])
    print(data['ms'])
    print(data['ms_abs'])
    print(data['mr'])
    print(data['mr_abs'])
    print(data['map'])
    print("")
    break 

The comment does not accurately describe the behavior of this API call. This doesn't remove any watches. By passing "false" here, it simply doesn't add any new ones.
The comment accurately describe behavior _CODE_ call. This remove watches. By passing "false" here, simply add new ones.
  public void removeTable(TableId tableId) throws KeeperException, InterruptedException {
    synchronized (tableStateCache) {
      tableStateCache.remove(tableId);
      zoo.recursiveDelete(zkRoot + Constants.ZTABLES + "/" + tableId + Constants.ZTABLE_STATE,
          NodeMissingPolicy.SKIP);
      String tableConfigPath = zkRoot + Constants.ZTABLES + "/" + tableId + Constants.ZTABLE_CONF;
      // The line below removes all the watches on the table configuration items from the Zookeeper
      // server
      zoo.getZooKeeper().getChildren(tableConfigPath, false);
      zoo.recursiveDelete(zkRoot + Constants.ZTABLES + "/" + tableId, NodeMissingPolicy.SKIP);
    }
  }

public void METHOD_1 ( TYPE_1 VAR_